[H2O AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) is an automated machine learning meta-algorithm that is part of the [H2O software library](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html#what-is-h2o). (It shold not be confused with [H2O DriverlessAI](https://www.h2o.ai/products/h2o-driverless-ai/), which is a commercial product and built from an entirely different code base.) H2O’s AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit. Stacked Ensembles – one based on all previously trained models, another one on the best model of each family – will be automatically trained on collections of individual models to produce highly predictive ensemble models which, in most cases, will be the top performing models in the AutoML Leaderboard.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/ieee-models-9503/"))

# Any results you write to the current directory are saved as output.

In [ ]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

In [ ]:
%%time
train = h2o.import_file("../input/ieee-models-9503/X_tr.csv")
test = h2o.import_file("../input/ieee-models-9503/X_te.csv")

In [ ]:
train.head(5)

In [ ]:
splits = train.split_frame(ratios=[0.75, 0.24], seed=1)  

training = splits[0]
validation = splits[1]

In [ ]:
x_train = training.columns[:-1]
y_train = 'isFraud'
# For binary classification, response should be a factor
training[y_train] = training[y_train].asfactor()

In [ ]:
x_val = validation.columns[:-1]
y_val = 'isFraud'
# For binary classification, response should be a factor
training[y_val] = training[y_val].asfactor()

In [ ]:
aml = H2OAutoML(seed=55, max_runtime_secs=30000)
aml.train(x=x_train, y=y_train, training_frame=training)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
h2o.remove(training)

In [ ]:
preds = aml.predict(test)
preds['p1'].as_data_frame().values.flatten().shape

In [ ]:
sample_submission = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')
sample_submission.shape

In [ ]:
sample_submission['isFraud'] = preds['p1'].as_data_frame().values
sample_submission.to_csv('submission.csv', index=False)